# Testing different SA methods 2/5
## nltk's sentiment analyzer
This code was partly taken from [here](www.nltk.org/howto/sentiment.html) TODO also attribute in thesis

In [1]:
# Imports
import nltk
import csv
import pickle
import random
import time

from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from gensim import corpora

# Ugly hackery, but necessary: stackoverflow.com/questions/4383571/importing-files-from-different-folder
import sys
sys.path.append('../../../')
    
from src.streaming import spark_functions

In [2]:
preprocess = spark_functions.preprocessor()
tokenize = spark_functions.tokenizer()

with open('./../../../data/interim/sanders_hydrated.csv') as csv_file:
    iterator = csv.reader(csv_file, delimiter=',')
    # Load the parts we need and preprocess as well as tokenize the text
    tweets = [(tokenize(preprocess(text)), sentiment) for (topic, sentiment, id, text) in iterator]
    # The tweets are ordered, so shuffle for the training/test split
    random.shuffle(tweets)

In [31]:
tweets[0]

4345

In [3]:
"There are %d tweets in this dataset" % len(tweets)

'There are 4347 tweets in this dataset'

In [5]:
# Split between training and test set
# 90% training, 10% test
threshold = int(0.9 * len(tweets))
train_tweets = tweets[:threshold]
test_tweets = tweets[threshold:]

In [6]:
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([nltk.sentiment.util.mark_negation(doc) for doc in train_tweets])

We use simple unigram word features, handling negation:

In [7]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
len(unigram_feats)

1109

In [8]:
sentim_analyzer.add_feat_extractor(nltk.sentiment.util.extract_unigram_feats, unigrams=unigram_feats)

We apply features to obtain a feature-value representation of our datasets:

In [9]:
training_set = sentim_analyzer.apply_features(train_tweets)
test_set = sentim_analyzer.apply_features(test_tweets)

We can now train our classifier on the training set, and subsequently output the evaluation results:

In [10]:
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)
# TODO check source (sadly the dataset cannot be downloaded)
# nltk.sentiment.util.demo_tweets(trainer)

Training classifier


In [11]:
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Evaluating NaiveBayesClassifier results...
Accuracy: 0.6528735632183909
F-measure [irrelevant]: 0.779874213836478
F-measure [negative]: 0.5333333333333333
F-measure [neutral]: 0.6473684210526315
F-measure [positive]: 0.26865671641791045
Precision [irrelevant]: 0.7469879518072289
Precision [negative]: 0.4745762711864407
Precision [neutral]: 0.6684782608695652
Precision [positive]: 0.34615384615384615
Recall [irrelevant]: 0.8157894736842105
Recall [negative]: 0.6086956521739131
Recall [neutral]: 0.6275510204081632
Recall [positive]: 0.21951219512195122


In [41]:
# optional to save your classifier so you can load it elsewhere without having to rebuild training set every time
save_classifier = open("./../../models/naive_bayes/nltk_sentiment_analyzer_all_labels.pickle","wb")
pickle.dump(sentim_analyzer, save_classifier)
save_classifier.close()